# Calculate Diversity Scores to Optimize (The Final Objective)

* Refinement: Use an efficient search method, as calculating all pairwise scores for a large valid pool is computationally expensive.
* Key Action: Implement a Greedy Search.

    * Start with a large pool of high-confidence valid grids (e.g., 50,000 grids).

    * Use a Greedy Selection Algorithm to iteratively choose the 100 grids that maximize the overall mean pairwise Hamming distance. This is much faster and more effective than trying random sets.